# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing graphlab functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in graphlab functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Fire up graphlab create

In [10]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.linear_model import LinearRegression

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [7]:
path = str(Path('.').absolute())

In [8]:
sales = pd.read_csv(path + '/kc_house_data.csv')
train_data = pd.read_csv(path +'/kc_house_train_data.csv')
test_data = pd.read_csv(path +'/kc_house_test_data.csv')

# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let GraphLab Create pick a random seed for you).  

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [21]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] 
X = train_data[example_features]
y = train_data['price']
example_model = LinearRegression()
example_model.fit(X,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [22]:
example_weight_summary = example_model.coef_
print(example_weight_summary)

[   315.40669062 -65081.88711588   6942.16598637]


# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing graphlab create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [23]:
example_predictions = example_model.predict(X)
print(example_predictions[0])# should be 271789.505878

271789.26537996973


# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [27]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    pred = model.predict(data)
    # Then compute the residuals/errors
    res = outcome-pred
    # Then square and add them up
    RSS = np.sum(res**2)
    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [43]:
X = test_data[example_features]
y = test_data['price']
rss_example_train = get_residual_sum_of_squares(example_model, X, y)
print(rss_example_train) # should be 2.7376153833e+14

273761940583133.75


# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [31]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [34]:
train_data['bedrooms_squared'] = train_data['bedrooms']**2
test_data['bedrooms_squared'] = test_data['bedrooms']**2

0

In [71]:
# create the remaining 3 features in both TEST and TRAIN data
train_data['bed_bath_rooms'] = train_data['bedrooms']*train_data['bathrooms']
test_data['bed_bath_rooms'] = test_data['bedrooms']*test_data['bathrooms']

train_data['log_sqft_living'] = np.log(train_data['sqft_living'])
test_data['log_sqft_living'] = np.log(test_data['sqft_living'])

train_data['lat_plus_long'] = train_data['lat'] + train_data['long']
test_data['lat_plus_long'] = test_data['lat'] + test_data['long']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

In [72]:
m_br_sq = np.mean(test_data['bedrooms_squared'])
m_bb = np.mean(test_data['bed_bath_rooms'])
m_log_sqft = np.mean(test_data['log_sqft_living'])
m_latlong = np.mean(test_data['lat_plus_long'])


print('means are %.2f,%.2f, %.2f,%.2f' %  (m_br_sq, m_bb, m_log_sqft, m_latlong))

means are 12.45,7.50, 7.55,-74.65


# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [44]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using graphlab.linear_regression.create() and look at the value of the weights/coefficients:

In [45]:
# Learn the three models: (don't forget to set validation_set = None)
y = train_data['price']
X1 = train_data[model_1_features]
X2 = train_data[model_2_features]
X3 = train_data[model_3_features]

model_1  = LinearRegression().fit(X1,y)
model_2  = LinearRegression().fit(X2,y)
model_3  = LinearRegression().fit(X3,y)

In [48]:
# Examine/extract each model's coefficients:
coeff_1 = model_1.coef_
coeff_2 = model_2.coef_
coeff_3 = model_3.coef_

print(coeff_1[2])
print(coeff_2[2])


15706.742082734629
-71461.30829275958


**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**
*positive*

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**
*negative*

Think about what this means.

*this mean that with the addition of the new feature 'bed_bathrooms' the weight for bathrooms severly changed. possibly because this feature is included in the added feature.*

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [55]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
rss_train_1 = get_residual_sum_of_squares(model_1, X1, y)
rss_train_2 = get_residual_sum_of_squares(model_2, X2, y)
rss_train_3 = get_residual_sum_of_squares(model_3, X3, y)

print(rss_train_1,rss_train_2,rss_train_3)
rss_train_2 - rss_train_3

967879963049545.5 958419635074069.8 903436455050477.6


54983180023592.125

**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

model_3 --> yes, more features means more accurate predictions on the same data

Now compute the RSS on on TEST data for each of the three models.

In [57]:
# Compute the RSS on TESTING data for each of the three models and record the values:
X1_test = test_data[model_1_features]
X2_test = test_data[model_2_features]
X3_test = test_data[model_3_features]
y_test = test_data.price

rss_test_1 = get_residual_sum_of_squares(model_1, X1_test, y_test)
rss_test_2 = get_residual_sum_of_squares(model_2, X2_test, y_test)
rss_test_3 = get_residual_sum_of_squares(model_3, X3_test, y_test)

print(rss_test_1, rss_test_2, rss_test_3)

225500469795490.03 223377462976466.94 3.860655825849427e+17


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected? Think about the features that were added to each model from the previous.

model 2: not really, i expected model 1 to be lowest, because it was the simepelest model that did not include nonesense features. However, thinking of it the 'bed_badrooms' feature might be a good predictor!

In [60]:
rss_test_1 - rss_test_2

2123006819023.0938